In [1]:
import sys; sys.path.insert(0, '../ia342-dynamic-programming/') # add parent folder path where lib folder

import sys; sys.path.insert(0, '../ia342-dynamic-programming/') # add parent folder path where lib folder

# IA342 Tópicos em Otimização de Sistemas III
## Programação Dinâmica, Variações e Aproximações


### Lista de Exercícios e Trabalhos

#### Lista 1


Para a solução da Lista 1 foi criada uma classe abstrata, [DiscreteDynamicProblem](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/discreteDynamicProblem.py), com alguns métodos que precisam ser implementados conforme o problema que deseja-se resolver, um método de solve e um que calcula a trajetória ótima.

O método de [solve](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/discreteDynamicProblem.py#L137) foi implementado como pode ser visto abaixo:

---
```
def solve(self):
        """
        Solves the optimality recursive equation using a backward strategy
        """
        for xk in self.state(self.numberOfStages()):
            self.__F[self.numberOfStages(), xk] = self.finalStateCost(xk)
        
        for k in self.__stagesGenerator(): #n-1 to 0
            for xk in self.state(k):
                F_aux = self.inf()
                u_aux = None
                # Calculates the best decision on stage k and state uk
                for uk in self.decision(k):
                    xk_next = self.transitionFunction(k, xk, uk)
                    F_aux_uk = self.elementaryCost(k, xk, uk) + self.F(k+1, xk_next)

                    if F_aux > F_aux_uk:
                        F_aux = F_aux_uk
                        u_aux = uk
                self.__F[k, xk] = F_aux
                self.__policy[k, xk] = u_aux
```
---

Além disso foi implementado um [método](https://github.com/seoruosa/ia342-dynamic-programming/blob/master/dypro/dypro/discreteDynamicProblem.py#L159) que calcula a trajetória ótima, dado um stado inicial:

---

```
def optimalTrajectory(self, initialState:np.array):
        """
        Calculates the optimal trajectory given a initial state

        Args:
            initialState (np.array): initial state to calculate the optimal trajectory

        Returns:
            u_optimal: states of the optimal trajectory
            policy_optimal: decisions for the optimal trajectory
        """
        u_optimal = [initialState]
        policy_optimal = list()
        for k in range(self.numberOfStages()):
            policy_optimal.append(self.policy(k, u_optimal[-1]))
            u_optimal.append(self.transitionFunction(k, u_optimal[-1], policy_optimal[-1]))
        
        return (u_optimal, policy_optimal, )
```

In [4]:
from dypro.dypro.discreteDynamicProblem import DiscreteDynamicProblem
from dypro.dypro.discreteDynamicProblem import generatorFromLIst
import numpy as np

##### Exercício 1 - Problema da fábrica de turbinas

Para a solução deste exercício foi necessário implementar os métodos abaixo, utilizando o método de solve da classe que ela implementa, conforme pode ser visto abaixo:

In [6]:
class SimpleProductionProblem(DiscreteDynamicProblem):
    def __init__(self, numberOfStages:int, initialState:np.array, demand, feasibleDecisions, feasibleStates, 
                finalStateCost, productionCost, currentStateCost, inf=np.inf, F=dict(), policy=dict()):
        self.__initialState = initialState
        self.__demand = demand
        self.__feasibleDecisions = feasibleDecisions
        self.__feasibleStates = feasibleStates
        self.__finalStateCost = finalStateCost
        self.__productionCost = productionCost
        self.__currentStateCost = currentStateCost
        super().__init__(numberOfStages, inf=inf, F=F, policy=policy)
        
    def state(self, k:int) -> np.array:
        return self.__feasibleStates(k)

    def decision(self, k:int) -> np.array:
        return self.__feasibleDecisions(k)

    def elementaryCost(self, k:int, state:np.array, decision:np.array) -> float:
        return self.__productionCost(k, decision) + self.__currentStateCost(k, state)

    def finalStateCost(self, state:np.array) -> float:
        return self.__finalStateCost(state)

    def transitionFunction(self, k:int, state:np.array, decision:np.array) -> np.array:
        return state + decision - self.__demand(k)

    def initialState(self) -> np.array:
        return self.__initialState
    

if __name__ == '__main__':
    
    STAGES = 4
    demandStage = lambda k: [2,1,1,0][k]
    stockCost = lambda k, state: [0, 3, 7][state] if state<3 else np.inf
    productionCost = lambda k, decision: [10, 17, 20][decision]
    stateSpace = lambda k: generatorFromLIst([0, 1, 2])
    productionSpace = lambda k: generatorFromLIst([0, 1, 2])
    finalState = lambda state: 0 if state==1 else np.inf
    initialStock = 1
    
    dp = SimpleProductionProblem(STAGES, initialStock, demandStage, productionSpace, stateSpace, 
                                 finalState, productionCost, stockCost)
    
    dp.solve()
    u_optimal, policy_optimal = dp.optimalTrajectory(initialStock)

    print(f"states of the optimal trajectory: {u_optimal}")
    print(f"Decisions for the optimal trajectory: {policy_optimal}")
    print(f"Cost of the optimal trajectory: {dp.F(0, initialStock)}")

states of the optimal trajectory: [1, 1, 0, 1, 1]
Decisions for the optimal trajectory: [2, 0, 2, 0]
Cost of the optimal trajectory: 69


##### Exercício 2 - Planejamento de produção com retardos

In [10]:
from dypro.dypro.discreteDynamicProblem import DiscreteDynamicProblem
from dypro.dypro.discreteDynamicProblem import generatorFromLIst, generatorFromLists
import numpy as np

class DelayProducerPlanning(DiscreteDynamicProblem):
    def __init__(self, numberOfStages:int, initialState:np.array, demand, feasibleDecisions, feasibleStates, 
                finalStateCost, productionCost, currentStateCost, changeStateCost, inf=np.inf, 
                 F=dict(), policy=dict()):
        self.__initialState = initialState
        self.__demand = demand
        self.__feasibleDecisions = feasibleDecisions
        self.__feasibleStates = feasibleStates
        self.__finalStateCost = finalStateCost
        self.__productionCost = productionCost
        self.__currentStateCost = currentStateCost
        self.__changeStateCost = changeStateCost
        super().__init__(numberOfStages, inf=inf, F=F, policy=policy)
        
    def state(self, k:int) -> np.array:
        return self.__feasibleStates(k)

    def decision(self, k:int) -> np.array:
        return self.__feasibleDecisions(k)

    def elementaryCost(self, k:int, state:np.array, decision:np.array) -> float:
        return self.__productionCost(k, decision) + self.__currentStateCost(k, state) + \
                self.__changeStateCost(k, state, decision)

    def finalStateCost(self, state:np.array) -> float:
        return self.__finalStateCost(state)

    def transitionFunction(self, k:int, state:np.array, decision:np.array) -> np.array:
        return (state[0] + decision - self.__demand(k), decision)

    def initialState(self) -> np.array:
        return self.__initialState
    

if __name__ == '__main__':
    
    STAGES = 3
    demandStage = lambda k: [2,4,1][k]
    stockCost = lambda k, state: state[0] if state[0]<=4 else np.inf
    changeProductionCost = lambda k, state, decision: max(decision - state[1], state[1] - decision)*2
    productionCost = lambda k, decision: ([3, 5, 3][k])*decision
    stateSpace = lambda k: generatorFromLists([0, 1, 2, 3, 4], [0, 1, 2, 3])
    productionSpace = lambda k: generatorFromLIst([0, 1, 2, 3])

    finalState = lambda state: 0
    initialState = (1, 1)
    
    dp = DelayProducerPlanning(STAGES, initialState, demandStage, productionSpace, stateSpace, 
                               finalState, productionCost, stockCost, changeProductionCost)
    dp.solve()
    u_optimal, policy_optimal = dp.optimalTrajectory(initialState)

    print(f"states of the optimal trajectory: {u_optimal}")
    print(f"Decisions for the optimal trajectory: {policy_optimal}")
    print(f"Cost of the optimal trajectory: {dp.F(0, initialState)}")

states of the optimal trajectory: [(1, 1), (2, 3), (0, 2), (0, 1)]
Decisions for the optimal trajectory: [3, 2, 1]
Cost of the optimal trajectory: 33
